<a href="https://colab.research.google.com/github/Educat8n/AI-Development-Oxford/blob/main/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Defining a simple neural network in TensorFlow 2.0

* We use TensorFlow 2.0 to define a network that recognizes MNIST handwritten digits. 
* We start with a very simple neural network and then progressively improve it.
* Load the dataset.
```
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
```
* X_train is used for fine-tuning our net, and test sets, X_test, used for assessing the performance. 
* Data is converted into float32 to use 32-bit precision when training a neural network and normalized to the range [0,1]. 
```
#normalize in [0,1]
X_train, X_test = X_train / 255.0, X_test / 255.0
#X_train is 60000 rows of 28x28 values --> reshaped in 60000 x 784
RESHAPED = 784
#
X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
```
* Perform a one-hot encoding on labels Y_train and Y_test.
```
# One-hot representation of the labels.
Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES) Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)
```




In [1]:
from tensorflow import keras
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
print(X_train.shape, Y_train.shape, 'train samples')
print(X_test.shape, Y_test.shape, 'test samples')

(60000, 28, 28) (60000,) train samples
(10000, 28, 28) (10000,) test samples


In [2]:
%%writefile mnist0.py
### MNIST Ver 0

import tensorflow as tf
import numpy as np
from tensorflow import keras

# for reproducibility
np.random.seed(1671)

# network and training
EPOCHS = 50
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10   # number of outputs = number of digits
N_HIDDEN = 128
VALIDATION_SPLIT=0.2 # how much TRAIN is reserved for VALIDATION

# loading MNIST dataset
# verify
# the split between train and test is 60,000, and 10,000 respectly 
# one-hot is automatically applied
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


#normalize in [0,1]
X_train, X_test = X_train / 255.0, X_test / 255.0
#X_train is 60000 rows of 28x28 values --> reshaped in 60000 x 784
RESHAPED = 784
#
X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
print(Y_train[1])
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# One-hot representation of the labels.
Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES) 
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(NB_CLASSES,
   		input_shape=(RESHAPED,), kernel_initializer='zeros',
   		name='dense_layer', activation='softmax'))

# summary of the model
model.summary()

# compiling the model
model.compile(optimizer='SGD', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#training the moodel
model.fit(X_train, Y_train,
		batch_size=BATCH_SIZE, epochs=EPOCHS,
		verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

#evalute the model
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)

# making prediction
#predictions = model.predict(X_test)

Overwriting mnist0.py


In [3]:
%run mnist0.py

60000 train samples
10000 test samples
0
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_layer (Dense)         (None, 10)                7850      
                                                                 
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
375/375 [==============================] - 3s 7ms/step - loss: 1.3527 - accuracy: 0.7781 - val_loss: 0.8779 - val_accuracy: 0.8438
Epoch 2/50
375/375 [==============================] - 2s 5ms/step - loss: 0.7803 - accuracy: 0.8424 - val_loss: 0.6476 - val_accuracy: 0.8642
Epoch 3/50
375/375 [==============================] - 2s 5ms/step - loss: 0.6349 - accuracy: 0.8574 - val_loss: 0.5546 - val_accuracy: 0.8753
Epoch 4/50
375/375 [==============================] - 2s 6ms/step - loss: 0.5645 - accuracy: 0.8657 - val_loss: 0.5

# Improving Model Performance- Hyperparameter Tuning

* Model Architecture
  * Change Number of hidden layers
  * Change number of hidden units/neurons


* Epochs
* Batch-Size
```
model.fit(X_train, Y_train,
		batch_size=BATCH_SIZE, epochs=EPOCHS,
		verbose=VERBOSE, validation_split=VALIDATION_SPLIT)
    ```

* Optimizer
 * SGD
 * Adam
 * Adadelta
```
model.compile(optimizer='SGD',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
```
* Learning Rate
```
from tensorflow.keras.optimizers import SGD
OPTMIZER = SGD(lr=0.001)
:
:
:
# compiling the model
model.compile(optimizer=OPTMIZER, 
              loss='categorical_crossentropy',
              metrics=['accuracy'])
```

* Regularization
 * L1/L2
 * Dropout Layer
 * Batchnormalization Layer


## Change Number of hidden layers.
```
# Build the model.
model = tf.keras.models.Sequential() 
model.add(keras.layers.Dense(N_HIDDEN,
             input_shape=(RESHAPED,),name='dense_layer', activation='relu')) 
model.add(keras.layers.Dense(N_HIDDEN,name='dense_layer_2', activation='relu')) 
model.add(keras.layers.Dense(NB_CLASSES,name='dense_layer_3', activation='softmax'))
# Summary of the model. 
model.summary()
```


In [4]:
%%writefile mnist1.py
### MNIST Ver 1

import tensorflow as tf
import numpy as np
from tensorflow import keras

# for reproducibility
np.random.seed(1671)

# network and training
EPOCHS = 50
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10   # number of outputs = number of digits
N_HIDDEN = 128
VALIDATION_SPLIT=0.2 # how much TRAIN is reserved for VALIDATION

# loading MNIST dataset
# verify
# the split between train and test is 60,000, and 10,000 respectly 
# one-hot is automatically applied
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


#normalize in [0,1]
X_train, X_test = X_train / 255.0, X_test / 255.0
#X_train is 60000 rows of 28x28 values --> reshaped in 60000 x 784
RESHAPED = 784
#
X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# One-hot representation of the labels.
Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES) 
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

model = tf.keras.models.Sequential() 
model.add(keras.layers.Dense(N_HIDDEN,
             input_shape=(RESHAPED,),name='dense_layer_1', activation='relu')) 
model.add(keras.layers.Dense(N_HIDDEN,name='dense_layer_2', activation='relu')) 
model.add(keras.layers.Dense(NB_CLASSES,name='dense_layer_3', activation='softmax'))

# summary of the model
model.summary()

# compiling the model
model.compile(optimizer='SGD', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#training the moodel
model.fit(X_train, Y_train,
		batch_size=BATCH_SIZE, epochs=EPOCHS,
		verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

#evalute the model
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)


Overwriting mnist1.py


In [5]:
%run mnist1.py

60000 train samples
10000 test samples
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_layer_1 (Dense)       (None, 128)               100480    
                                                                 
 dense_layer_2 (Dense)       (None, 128)               16512     
                                                                 
 dense_layer_3 (Dense)       (None, 10)                1290      
                                                                 
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
375/375 [==============================] - 2s 5ms/step - loss: 1.4708 - accuracy: 0.6244 - val_loss: 0.7513 - val_accuracy: 0.8412
Epoch 2/50
375/375 [==============================] - 2s 5ms/step - loss: 0.5959 - accuracy: 0.8531 - val_loss: 0.4573 - val_accura

## Change number of hidden units

```
N_HIDDEN = 64
:
:
# Build the model.
model = tf.keras.models.Sequential() 
model.add(keras.layers.Dense(N_HIDDEN,
             input_shape=(RESHAPED,),name='dense_layer', activation='relu')) 
model.add(keras.layers.Dense(N_HIDDEN,name='dense_layer_2', activation='relu')) 
model.add(keras.layers.Dense(NB_CLASSES,name='dense_layer_3', activation='softmax'))
# Summary of the model. 
model.summary()
```


In [6]:
%%writefile mnist2.py
### MNIST Ver 2

import tensorflow as tf
import numpy as np
from tensorflow import keras

# for reproducibility
np.random.seed(1671)

# network and training
EPOCHS = 50
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10   # number of outputs = number of digits
N_HIDDEN = 64
VALIDATION_SPLIT=0.2 # how much TRAIN is reserved for VALIDATION

# loading MNIST dataset
# verify
# the split between train and test is 60,000, and 10,000 respectly 
# one-hot is automatically applied
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


#normalize in [0,1]
X_train, X_test = X_train / 255.0, X_test / 255.0
#X_train is 60000 rows of 28x28 values --> reshaped in 60000 x 784
RESHAPED = 784
#
X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# One-hot representation of the labels.
Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES) 
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

model = tf.keras.models.Sequential() 
model.add(keras.layers.Dense(N_HIDDEN,
             input_shape=(RESHAPED,),name='dense_layer_1', activation='relu')) 
model.add(keras.layers.Dense(N_HIDDEN,name='dense_layer_2', activation='relu')) 
model.add(keras.layers.Dense(NB_CLASSES,name='dense_layer_3', activation='softmax'))

# summary of the model
model.summary()

# compiling the model
model.compile(optimizer='SGD', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#training the moodel
model.fit(X_train, Y_train,
		batch_size=BATCH_SIZE, epochs=EPOCHS,
		verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

#evalute the model
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)


Overwriting mnist2.py


In [7]:
%run mnist2.py

60000 train samples
10000 test samples
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_layer_1 (Dense)       (None, 64)                50240     
                                                                 
 dense_layer_2 (Dense)       (None, 64)                4160      
                                                                 
 dense_layer_3 (Dense)       (None, 10)                650       
                                                                 
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
375/375 [==============================] - 2s 4ms/step - loss: 1.5767 - accuracy: 0.5627 - val_loss: 0.8543 - val_accuracy: 0.8065
Epoch 2/50
375/375 [==============================] - 1s 4ms/step - loss: 0.6607 - accuracy: 0.8334 - val_loss: 0.4861 - val_accuracy

## Change Model Fit Parameters
* Change Epochs
* Change Batch Size
```
# network and training
EPOCHS = 50
BATCH_SIZE = 128
```


In [8]:
%%writefile mnist3.py
### MNIST Ver 3

import tensorflow as tf
import numpy as np
from tensorflow import keras

# for reproducibility
np.random.seed(1671)

# network and training
EPOCHS = 75
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10   # number of outputs = number of digits
N_HIDDEN = 64
VALIDATION_SPLIT=0.2 # how much TRAIN is reserved for VALIDATION

# loading MNIST dataset
# verify
# the split between train and test is 60,000, and 10,000 respectly 
# one-hot is automatically applied
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


#normalize in [0,1]
X_train, X_test = X_train / 255.0, X_test / 255.0
#X_train is 60000 rows of 28x28 values --> reshaped in 60000 x 784
RESHAPED = 784
#
X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# One-hot representation of the labels.
Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES) 
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

model = tf.keras.models.Sequential() 
model.add(keras.layers.Dense(N_HIDDEN,
             input_shape=(RESHAPED,),name='dense_layer_1', activation='relu')) 
model.add(keras.layers.Dense(N_HIDDEN,name='dense_layer_2', activation='relu')) 
model.add(keras.layers.Dense(NB_CLASSES,name='dense_layer_3', activation='softmax'))

# summary of the model
model.summary()

# compiling the model
model.compile(optimizer='SGD', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#training the moodel
model.fit(X_train, Y_train,
		batch_size=BATCH_SIZE, epochs=EPOCHS,
		verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

#evalute the model
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)


Overwriting mnist3.py


In [9]:
%run mnist3.py

60000 train samples
10000 test samples
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_layer_1 (Dense)       (None, 64)                50240     
                                                                 
 dense_layer_2 (Dense)       (None, 64)                4160      
                                                                 
 dense_layer_3 (Dense)       (None, 10)                650       
                                                                 
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________
Epoch 1/75
375/375 [==============================] - 2s 4ms/step - loss: 1.5882 - accuracy: 0.5784 - val_loss: 0.9002 - val_accuracy: 0.7902
Epoch 2/75
375/375 [==============================] - 2s 5ms/step - loss: 0.6888 - accuracy: 0.8254 - val_loss: 0.5141 - val_accuracy

## Change Optimizer
```
model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
```



In [10]:
%%writefile mnist4.py
### MNIST Ver 4

import tensorflow as tf
import numpy as np
from tensorflow import keras

# for reproducibility
np.random.seed(1671)

# network and training
EPOCHS = 50
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10   # number of outputs = number of digits
N_HIDDEN = 64
VALIDATION_SPLIT=0.2 # how much TRAIN is reserved for VALIDATION

# loading MNIST dataset
# verify
# the split between train and test is 60,000, and 10,000 respectly 
# one-hot is automatically applied
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


#normalize in [0,1]
X_train, X_test = X_train / 255.0, X_test / 255.0
#X_train is 60000 rows of 28x28 values --> reshaped in 60000 x 784
RESHAPED = 784
#
X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# One-hot representation of the labels.
Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES) 
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

model = tf.keras.models.Sequential() 
model.add(keras.layers.Dense(N_HIDDEN,
             input_shape=(RESHAPED,),name='dense_layer_1', activation='relu')) 
model.add(keras.layers.Dense(N_HIDDEN,name='dense_layer_2', activation='relu')) 
model.add(keras.layers.Dense(NB_CLASSES,name='dense_layer_3', activation='softmax'))

# summary of the model
model.summary()

# compiling the model
model.compile(optimizer='SGD', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#training the moodel
model.fit(X_train, Y_train,
		batch_size=BATCH_SIZE, epochs=EPOCHS,
		verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

#evalute the model
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)


Overwriting mnist4.py


In [11]:
%run mnist4.py

60000 train samples
10000 test samples
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_layer_1 (Dense)       (None, 64)                50240     
                                                                 
 dense_layer_2 (Dense)       (None, 64)                4160      
                                                                 
 dense_layer_3 (Dense)       (None, 10)                650       
                                                                 
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
375/375 [==============================] - 2s 4ms/step - loss: 1.4740 - accuracy: 0.6432 - val_loss: 0.7296 - val_accuracy: 0.8455
Epoch 2/50
375/375 [==============================] - 1s 4ms/step - loss: 0.5870 - accuracy: 0.8549 - val_loss: 0.4517 - val_accuracy

## Change Learning Rate

```
from tensorflow.keras.optimizers import SGD
OPTMIZER = SGD(lr=0.001)
:
:
:
# compiling the model
model.compile(optimizer=OPTMIZER, 
              loss='categorical_crossentropy',
              metrics=['accuracy'])
```


In [12]:
%%writefile mnist5.py
### MNIST Ver 5

import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.optimizers import SGD

OPTMIZER = SGD(lr=0.001)
# for reproducibility
np.random.seed(1671)

# network and training
EPOCHS = 50
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10   # number of outputs = number of digits
N_HIDDEN = 64
VALIDATION_SPLIT=0.2 # how much TRAIN is reserved for VALIDATION

# loading MNIST dataset
# verify
# the split between train and test is 60,000, and 10,000 respectly 
# one-hot is automatically applied
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


#normalize in [0,1]
X_train, X_test = X_train / 255.0, X_test / 255.0
#X_train is 60000 rows of 28x28 values --> reshaped in 60000 x 784
RESHAPED = 784
#
X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# One-hot representation of the labels.
Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES) 
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

model = tf.keras.models.Sequential() 
model.add(keras.layers.Dense(N_HIDDEN,
             input_shape=(RESHAPED,),name='dense_layer_1', activation='relu')) 
model.add(keras.layers.Dense(N_HIDDEN,name='dense_layer_2', activation='relu')) 
model.add(keras.layers.Dense(NB_CLASSES,name='dense_layer_3', activation='softmax'))

# summary of the model
model.summary()

# compiling the model
model.compile(optimizer=OPTMIZER, 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#training the moodel
model.fit(X_train, Y_train,
		batch_size=BATCH_SIZE, epochs=EPOCHS,
		verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

#evalute the model
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)


Overwriting mnist5.py


In [13]:
%run mnist5.py

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


60000 train samples
10000 test samples
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_layer_1 (Dense)       (None, 64)                50240     
                                                                 
 dense_layer_2 (Dense)       (None, 64)                4160      
                                                                 
 dense_layer_3 (Dense)       (None, 10)                650       
                                                                 
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
375/375 [==============================] - 2s 4ms/step - loss: 2.2868 - accuracy: 0.1214 - val_loss: 2.2151 - val_accuracy: 0.1903
Epoch 2/50
375/375 [==============================] - 2s 5ms/step - loss: 2.1554 - accuracy: 0.2563 - val_loss: 2.0829 - val_accuracy

## Add Regularization
 * L1/L2
  
  There are three different types of regularization used in machine learning:
  * L1 regularization (also known as LASSO): The complexity of the model is expressed as the sum of the absolute values of the weights.
  * L2 regularization (also known as Ridge): The complexity of the model is expressed as the sum of the squares of the weights
  * Elastic regularization: The complexity of the model is captured by a combination of the preceding two techniques
 ```
from tf.keras.regularizers import l2, activity_l2 
model.add(Dense(64, input_dim=64, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
 ```


## Add Special Layers
 * Dropout Layer
```
model.add(keras.layers.Dropout(DROPOUT))
```
 * Batchnormalization Layer
```
model.add(keras.layers.BatchNormalization()
```
